In [75]:
import numpy
from urllib.request import urlopen
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import random
import copy
import gzip
import json
import string

def parseData(fname):
    for l in open(fname):
        yield eval(l)
def feature_label(data):
    return data["fit"]
def feature_X(data):
    return [1,data['height'],data['weight'],int(data['rating'])]
def str_to_inch(s):
    s = s.replace('\"', '')
    height=s.split("\'")
    return int(height[0])*12+int(height[1])
def str_to_weight(s):
    s = s.replace('lbs', '')
    return int(s)


test_data = list(parseData("modcloth_final_data.json"))
#random.shuffle(data)
print(test_data[:10])




[{'item_id': '123373', 'waist': '29', 'size': 7, 'quality': 5, 'cup size': 'd', 'hips': '38', 'bra size': '34', 'category': 'new', 'bust': '36', 'height': '5ft 6in', 'user_name': 'Emily', 'length': 'just right', 'fit': 'small', 'user_id': '991571'}, {'item_id': '123373', 'waist': '31', 'size': 13, 'quality': 3, 'cup size': 'b', 'hips': '30', 'bra size': '36', 'category': 'new', 'length': 'just right', 'height': '5ft 2in', 'user_name': 'sydneybraden2001', 'fit': 'small', 'user_id': '587883'}, {'item_id': '123373', 'waist': '30', 'size': 7, 'quality': 2, 'cup size': 'b', 'shoe size': '9.00', 'bra size': '32', 'category': 'new', 'length': 'slightly long', 'height': '5ft 7in', 'user_name': 'Ugggh', 'fit': 'small', 'user_id': '395665'}, {'item_id': '123373', 'category': 'new', 'size': 21, 'quality': 5, 'user_name': 'alexmeyer626', 'length': 'just right', 'fit': 'fit', 'cup size': 'dd/e', 'user_id': '875643'}, {'item_id': '123373', 'category': 'new', 'size': 18, 'quality': 5, 'user_name': 'd

In [31]:
#f = open('renttherunway_final_data.json', "r")
data = list([json.loads(line) for line in open('renttherunway_final_data.json', 'r')])
    
data1=[]
data_test=[]
for i in data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        if i['rating'] != None:
            i['weight']=str_to_weight(i['weight'])
            i['height']=str_to_inch(i['height'])
            data1.append(i)

for i in test_data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        i['weight']=str_to_weight(i['weight'])
        i['height']=str_to_inch(i['height'])
        data_test.append(i)

In [32]:
#print(len(data_train),len(data))
print(len(data_test),len(test_data))    #different data sets cannot be test with training set

random.shuffle(data1)
data_train=data1[:len(data1)//10*9]
data_test=data1[len(data1)//10*9:]
print(len(data_train),len(data_test),len(data1))

0 82790
136152 15132 151284


In [11]:
print(type(data[0]))
for key in data[0]:
    print(key,':',data[0][key])



string1= "5'8\""
#print(data_train[1]['height'])
#print(str_to_inch(data_train[1]['height']))
    

<class 'dict'>
fit : fit
user_id : 420272
bust size : 34d
item_id : 2260466
weight : 137
rating : 10
rented for : vacation
review_text : An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.
body type : hourglass
review_summary : So many compliments!
category : romper
height : 68
size : 14
age : 28
review_date : April 20, 2016


In [12]:
#print(test_data[:1])
l=[feature_height(d) for d in data_train]
dic= defaultdict(int)
print(l[0])
for i in l:                                       #num of fit/small/large
    j=str(i)
    dic[j]+=1

p=[d['category'] for d in data_train]             #num of items per category
for i in p:
    j=str(i)
    dic[j]+=1
    
print(dic)

['fit']
defaultdict(<class 'int'>, {"['fit']": 111464, "['small']": 20330, "['large']": 19541, 'romper': 2517, 'gown': 34473, 'dress': 72916, 'sheath': 15189, 'leggings': 95, 'sweater': 948, 'jacket': 1909, 'shirtdress': 583, 'jumpsuit': 4159, 'maxi': 2678, 'shift': 4137, 'top': 3923, 'shirt': 220, 'mini': 1438, 'skirt': 1241, 'pants': 340, 'suit': 108, 'coat': 792, 'blouse': 504, 'trench': 20, 'cape': 69, 'bomber': 103, 'blazer': 633, 'vest': 222, 'duster': 12, 'ballgown': 13, 'tank': 145, 'poncho': 34, 'frock': 170, 'tunic': 137, 'cardigan': 203, 'down': 387, 'culottes': 163, 'midi': 46, 'legging': 81, 'print': 90, 'pant': 85, 'knit': 42, 'culotte': 60, 'sweatshirt': 104, 'peacoat': 33, 'trouser': 50, 'kaftan': 13, 'overalls': 6, 'jogger': 4, 'tee': 20, 'combo': 8, 'henley': 7, 'blouson': 11, 'pullover': 46, 'turtleneck': 23, 'trousers': 12, 'overcoat': 1, 'hoodie': 11, 't-shirt': 11, 'caftan': 4, 'tight': 13, 'kimono': 24, 'cami': 12, 'for': 5, 'crewneck': 1, 'skirts': 5, 'parka': 1

In [13]:
#possible missing values:

#bust size
#weight
#rating
#review_text
#height


In [14]:
usersPeritem = defaultdict(set)
itemPerUser = defaultdict(set)
for d in data:
    user,item = d['user_id'], d['item_id']
    usersPeritem[item].add(user)
    itemPerUser[user].add(item)
    






In [18]:
for i in itemPerUser:
    print(len(itemPerUser[i]))
print(len(itemPerUser))          #

6
1
1
1
7
1
29
1
1
5
1
1
3
2
3
3
2
8
2
7
2
1
3
6
1
6
2
11
1
7
1
1
2
3
1
2
2
1
11
2
3
1
22
1
23
2
12
1
7
9
11
20
5
1
1
26
1
2
12
30
1
124
7
1
8
1
104
6
1
4
3
17
1
5
2
9
1
1
20
1
8
1
2
1
1
14
1
1
3
1
5
1
3
2
1
2
3
2
1
4
3
3
1
3
1
5
1
2
2
2
1
2
1
6
1
2
2
2
3
3
1
4
3
3
1
14
9
1
2
1
11
6
1
2
1
6
8
1
1
1
1
1
11
1
1
1
5
1
2
1
1
2
2
4
3
2
2
1
1
292
2
19
1
11
4
14
1
2
2
1
2
5
1
5
15
4
4
1
1
10
3
1
2
1
8
1
1
2
1
3
1
6
1
1
19
1
5
5
1
2
1
5
1
1
1
1
1
1
2
2
2
1
7
1
1
1
1
1
1
9
4
1
3
6
2
1
2
5
12
2
12
1
2
7
1
2
1
3
14
2
31
2
1
1
1
1
3
1
18
29
13
1
14
1
1
12
4
1
1
9
1
1
1
1
1
4
1
1
2
1
2
4
1
2
1
13
6
2
7
81
2
140
7
1
2
53
8
1
1
9
1
1
1
1
9
1
4
3
3
2
1
5
1
4
3
6
15
1
7
3
2
8
13
1
6
3
80
4
28
1
29
1
1
1
6
1
1
10
8
1
1
7
4
7
2
3
2
3
8
228
2
27
67
44
1
1
3
1
1
2
4
4
1
3
1
4
27
13
1
8
1
3
9
8
3
1
36
5
27
1
2
3
1
3
14
1
15
3
20
1
1
1
1
3
3
2
4
5
6
1
3
2
9
1
1
2
2
1
1
7
1
1
1
48
1
3
2
2
13
1
2
11
2
2
1
13
10
1
6
8
2
2
1
1
4
4
6
1
3
2
2
4
15
2
30
1
5
2
2
2
2
6
2
3
1
3
3
2
15
4
15
2
4
28
6
28
9
1
2
1
3
2
7
5


In [65]:
X_train=[feature_X(d) for d in data_train]
X_test=[feature_X(d) for d in data_test]

Y_train=[feature_label(d) for d in data_train]
Y_test=[feature_label(d) for d in data_test]

In [66]:
mod = linear_model.LogisticRegression(C=2.0,)
mod.fit(X_train, Y_train)




LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
pred_valid= mod.predict(X_test)
correct1 = pred_valid==Y_test

print("accuracy feature:weight,height,rating", sum(correct1)/len(correct1))
#print(len(correct1))
print('done..')

accuracy feature:weight,height,rating 0.73949246629659
done..


In [73]:


dic1= defaultdict(int)
dic2=defaultdict(int)

print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

['fit']
defaultdict(<class 'int'>, {'fit': 14803, 'small': 329}) defaultdict(<class 'int'>, {'fit': 11179, 'large': 1954, 'small': 1999})


In [69]:
print(type(Y_train[1]))

<class 'str'>


In [70]:
print([feature_label(d) for d in data_train[:5]])

['fit', 'fit', 'fit', 'fit', 'fit']


In [77]:
#try bag of words methods
sp = set(string.punctuation)
wordCount = defaultdict(int)
for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()



In [82]:
print(len(counts))

44034


In [83]:
from scipy.sparse import lil_matrix
word_num = len(counts)//10
words = [x[1] for x in counts[:word_num]]
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
filterword = [w for w in words if not w in stopwords]
#filterword = [w for w in words]
wordId = dict(zip(filterword, range(len(filterword))))
wordSet = set(filterword)
word_df = dict(zip(filterword, [0]*(len(filterword))))

for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    sets= set(tokens)
    for i in sets:
        if i in filterword:
            word_df[i]+=1
print('done..')

done..


In [86]:
def feature(datum):
    feat = [0]*len(filterword)
    r = ''.join([c for c in datum['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
    sets=set(tokens)
    for s in sets:
        if s in wordSet:
            feat[wordId[s]]=feat[wordId[s]]*word_df[s]

    feat.append(1)
    return feat

nf = len(feature(data1[0]))

X = lil_matrix((len(data1), nf))

for i in range(len(data1)):
    
    x = feature(data[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]

print('done..')

done..


In [87]:
Xtrain = X[:len(data1)//10*9]

Xtest = X[len(data1)//10*9:]



mod1 = linear_model.LogisticRegression(C=2.0)
mod1.fit(Xtrain, Y_train)
print('done')

done


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [89]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7370473169442242
done..


In [90]:
dic1= defaultdict(int)
dic2=defaultdict(int)

print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

['fit']
defaultdict(<class 'int'>, {'fit': 15093, 'small': 18, 'large': 21}) defaultdict(<class 'int'>, {'fit': 11179, 'large': 1954, 'small': 1999})
